In [ ]:
!pip install nomic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for nomic: filename=nomic-3.0.12-py3-none-any.whl size=41557 sha256=54e049ae5c2e13338b360e80b3187f13b868a6a2becbeb79d1df9d4f80953581
  Stored in directory: /root/.cache/pip/wheels/30/2c/0e/e559ff44f0f908cd996021832fb98ea169372e7d91a1b41795
Successfully built nomic


In [ ]:
! nomic login

                                  Authenticate with the Nomic API                                   
                                  https://atlas.nomic.ai/cli-login                                  
       Click the above link to retrieve your access token and then run `nomic login [token]`        


In [ ]:
! nomic login <login_token>

In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchain-openai chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Optional: LangSmith API keys
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "api_key"

## Document Loading

Let's test 3 interesting blog posts.

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("Yearly Medical Examination Report for a Young College Student.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)
doc_splits = text_splitter.split_documents(docs)
len(doc_splits)

5

## Splitting

Long context retrieval

In [ ]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
for d in doc_splits:
    print("The document is %s tokens" % len(encoding.encode(d.page_content)))

The document is 398 tokens
The document is 134 tokens
The document is 402 tokens
The document is 298 tokens
The document is 234 tokens


## Index

Nomic embeddings [here](https://docs.nomic.ai/reference/endpoints/nomic-embed-text).

In [ ]:
import os

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_nomic import NomicEmbeddings
from langchain_nomic.embeddings import NomicEmbeddings

In [ ]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever

VectorStoreRetriever(tags=['Chroma', 'NomicEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7cb131a757b0>)

In [ ]:
question = "fracture"
docs = vectorstore.similarity_search(question,k=4)
len(docs)
vectorstore.persist()

In [ ]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 19.8 MB/s eta 0:00:00


In [ ]:
from gpt4all import GPT4All
model = GPT4All("nous-hermes-llama2-13b.Q4_0.gguf")

100%|██████████| 7.37G/7.37G [02:12<00:00, 55.7MiB/s]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

context = ''
for i in range(4):
  context+="\n"+docs[i].page_content

question = "Summarize the person's information related to conjunctivitis, What could be the potential diagnose?"

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = template.format(context=context, question=question)

In [ ]:
output = model.generate(prompt, max_tokens=600)
print(output)

Answer: The person had a tailbone fracture three years ago due to a snowboarding accident. It was managed with pain medication and physical therapy, with full recovery and no complications.
